# Proyecto: Un proceso de ingeniería de características

Curso Ingeniería de Características

<i class="fa-duotone fa-cactus"></i>

`ID` para el indicador de los municipios.<br>
`FECHA` para las series de tiempo (formato YYYY-MM-DD)

In [20]:
import pandas as pd
import geopandas as gpd
import warnings
from cod_dict import * # Módulo para accesar a los diccionarios de códigos
warnings.filterwarnings('ignore')

### De archivo a dataframe

In [3]:
# Datos georreferenciados
raw_shapes_gdf=gpd.read_file('raw_data/geo_shp/municipal.shp')
# Datos de población
raw_pobl_df=pd.read_excel('raw_data/pob_son.xlsx')
# Datos de unidades económicas 
raw_denue15_df=pd.read_csv('raw_data/denue/denue15.csv')
raw_denue16_df=pd.read_csv('raw_data/denue/denue16.csv', low_memory=False)
raw_denue17_df=pd.read_csv('raw_data/denue/denue17.csv', low_memory=False)
raw_denue18_df=pd.read_csv('raw_data/denue/denue18.csv', low_memory=False)
raw_denue19_df=pd.read_csv('raw_data/denue/denue19.csv', low_memory=False, encoding='latin-1')
raw_denue20_df=pd.read_csv('raw_data/denue/denue20.csv', low_memory=False, encoding='latin-1')
# Datos de incidencia delictiva
raw_delitos15_df=pd.read_excel('raw_data/delitos/delitos15.xlsx')
raw_delitos16_df=pd.read_excel('raw_data/delitos/delitos16.xlsx')
raw_delitos17_df=pd.read_excel('raw_data/delitos/delitos17.xlsx')
raw_delitos18_df=pd.read_excel('raw_data/delitos/delitos18.xlsx')
raw_delitos19_df=pd.read_excel('raw_data/delitos/delitos19.xlsx')
raw_delitos20_df=pd.read_excel('raw_data/delitos/delitos20.xlsx')

## Datos georreferenciados

In [7]:
shapes_gdf=raw_shapes_gdf.copy(deep=True)
# Selección de columnas
shapes_gdf=shapes_gdf[['CVEGEO', 'NOMBRE', 'geometry']]
# Formato de nombre
shapes_gdf.rename(columns={'CVEGEO':'ID'}, inplace=True)
# Formato geoespacial
shapes_gdf.to_crs(4326, inplace=True)

## Datos de población

In [24]:
pobl_df=raw_pobl_df.copy(deep=True)
# Filtrado de resgistros
pobl_df=pobl_df[pobl_df['indicador']=='Población total']
pobl_df=pobl_df[pobl_df['cve_municipio']!=0] # La cve_municipio=0 es el total del estado, innecesario
# Eliminar columnas vacías o NaN
pobl_df.dropna(axis=1, how='all', inplace=True)
# Selección de columnas
pobl_df=pobl_df[['cve_municipio', '1995', '2000', '2005', '2010', '2020']]
# Formato de nombre
pobl_df.rename(columns={'cve_municipio':'ID'}, inplace=True)
# Reemplazo de NaN
pobl_df.fillna(value=0,inplace=True)
# Formato de ID
pobl_df['ID']=pobl_df['ID']+26000
pobl_df['ID']=pobl_df['ID'].astype(str)
# Variables numéricas a int64
pobl_df.iloc[:,1:]=pobl_df.iloc[:,1:].astype('int64')

## Datos de unidades económicas

In [9]:
def denue_cleaner(df, agno):
    df_op=df.copy(deep=True)
    # Variable contador
    df_op[agno]=1
    # Únicamente para el dataset denue-2015 
    if agno == '2015':
        # Selección de columnas
        df_op=df_op[['Código de la clase de actividad SCIAN','Clave municipio', agno]]
        # Formato de nombre
        df_op.rename(columns={'Código de la clase de actividad SCIAN':'COD_ACT',
            'Clave municipio':'ID'}, inplace=True)
    else:
        # Selección de columnas
        df_op=df_op[['codigo_act','cve_mun', agno]]
        # Formato de nombre
        df_op.rename(columns={'codigo_act':'COD_ACT',
            'cve_mun':'ID'}, inplace=True)
    # Codificación de las unidades por sector económico usando regex
    df_op['COD_ACT']=df_op['COD_ACT'].astype(str).str.replace('^11....','UE_AGRO',regex=True)
    df_op['COD_ACT']=df_op['COD_ACT'].astype(str).str.replace('^21....','UE_MIN',regex=True)
    df_op['COD_ACT']=df_op['COD_ACT'].astype(str).str.replace('^22....','UE_ENE',regex=True)
    df_op['COD_ACT']=df_op['COD_ACT'].astype(str).str.replace('^23....','UE_CON',regex=True)
    df_op['COD_ACT']=df_op['COD_ACT'].astype(str).str.replace('^3[1-3]....','UE_MAN',regex=True)
    df_op['COD_ACT']=df_op['COD_ACT'].astype(str).str.replace('^43....','UE_CMA',regex=True)
    df_op['COD_ACT']=df_op['COD_ACT'].astype(str).str.replace('^46....','UE_CME',regex=True)
    df_op['COD_ACT']=df_op['COD_ACT'].astype(str).str.replace('^4[8-9]....','UE_TRA',regex=True)
    df_op['COD_ACT']=df_op['COD_ACT'].astype(str).str.replace('^51....','UE_MM',regex=True)
    df_op['COD_ACT']=df_op['COD_ACT'].astype(str).str.replace('^52....','UE_BAN',regex=True)
    df_op['COD_ACT']=df_op['COD_ACT'].astype(str).str.replace('^53....','UE_INM',regex=True)
    df_op['COD_ACT']=df_op['COD_ACT'].astype(str).str.replace('^54....','UE_PCT',regex=True)
    df_op['COD_ACT']=df_op['COD_ACT'].astype(str).str.replace('^55....','UE_COR',regex=True)
    df_op['COD_ACT']=df_op['COD_ACT'].astype(str).str.replace('^56....','UE_MRR',regex=True)
    df_op['COD_ACT']=df_op['COD_ACT'].astype(str).str.replace('^61....','UE_EDU',regex=True)
    df_op['COD_ACT']=df_op['COD_ACT'].astype(str).str.replace('^62....','UE_SAL',regex=True)
    df_op['COD_ACT']=df_op['COD_ACT'].astype(str).str.replace('^71....','UE_REC',regex=True)
    df_op['COD_ACT']=df_op['COD_ACT'].astype(str).str.replace('^72....','UE_ATA',regex=True)
    df_op['COD_ACT']=df_op['COD_ACT'].astype(str).str.replace('^81....','UE_OSE',regex=True)
    df_op['COD_ACT']=df_op['COD_ACT'].astype(str).str.replace('^93....','UE_GUB',regex=True)
    # Agrupación por sector económico
    df_op=df_op.groupby(by=['ID','COD_ACT']).sum().rename_axis(columns = None).reset_index()
    # Pivot de los sectores a variables
    df_op=df_op.pivot(index='ID',columns='COD_ACT',values=agno).rename_axis(columns = None).reset_index()
    # Reemplazo de NaN generados
    df_op.fillna(value=0,inplace=True)
    # Variables numéricas a int64 
    df_op.iloc[:,-20:]=df_op.iloc[:,-20:].astype('int64')
    # Variable identidicador de año en formato datetime
    df_op['FECHA']='31-12-'+agno
    df_op['FECHA']=pd.to_datetime(df_op['FECHA'], dayfirst=True)
    # Formato de ID
    df_op['ID']=df_op['ID']+26000
    df_op['ID']=df_op['ID'].astype(str)
    return df_op

In [10]:
denue15_df=denue_cleaner(raw_denue15_df, '2015')
denue16_df=denue_cleaner(raw_denue16_df, '2016')
denue17_df=denue_cleaner(raw_denue17_df, '2017')
denue18_df=denue_cleaner(raw_denue18_df, '2018')
denue19_df=denue_cleaner(raw_denue19_df, '2019')
denue20_df=denue_cleaner(raw_denue20_df, '2020')

## Datos de incidencia delictiva

In [11]:
def delitos_cleaner(df,agno):
    # Variables para renombrar las columnas de los meses
    ene, feb, mar=f'31-01-{agno}', f'28-02-{agno}', f'31-03-{agno}'
    abr, may, jun=f'30-04-{agno}', f'31-05-{agno}', f'30-06-{agno}'
    jul, ago, sep=f'31-07-{agno}', f'31-08-{agno}', f'30-09-{agno}'
    oct, nov, dic=f'31-10-{agno}', f'30-11-{agno}', f'31-12-{agno}'

    df_op=df.copy(deep=True)
    # Filtrado de resgistros
    df_op=df_op[df_op['Entidad']=='Sonora']
    df_op=df_op[df_op['Municipio']!='No Especificado']
    df_op=df_op[df_op['Municipio']!='Otros Municipios']
    # Formato de nombre
    df_op.rename(columns={'Cve. Municipio':'ID', 'Bien jurídico afectado':'DELITO',
        'Enero':ene, 'Febrero':feb, 'Marzo':mar,
        'Abril':abr, 'Mayo':may, 'Junio':jun,
        'Julio':jul, 'Agosto':ago, 'Septiembre':sep,
        'Octubre':oct, 'Noviembre':nov, 'Diciembre':dic}, inplace=True)
    # Selección de columnas
    df_op=df_op[['ID', 'DELITO', ene, feb, mar, abr, may,
        jun, jul, ago, sep, oct, nov, dic]]
    # Agrupación por ID y clasificación de delito
    df_op=df_op.groupby(by=['ID','DELITO']).sum().rename_axis(columns = None).reset_index()
    # Columnas de meses a variables tipo datetime
    df_op=df_op.melt(id_vars=['ID','DELITO'], var_name='FECHA')
    df_op['FECHA']=pd.to_datetime(df_op['FECHA'], dayfirst=True)
    # Pivot de los delitos a variables
    df_op=df_op.pivot(index=['ID','FECHA'], columns='DELITO', values='value').rename_axis(columns = None).reset_index()
    # Formato de nombres de delitos según su clave
    df_op.rename(columns=del_dict,inplace=True)
    # Formato de ID
    df_op['ID']=df_op['ID'].astype(str)
    return df_op

In [12]:
delitos15_df=delitos_cleaner(raw_delitos15_df,'2015')
delitos16_df=delitos_cleaner(raw_delitos16_df,'2016')
delitos17_df=delitos_cleaner(raw_delitos17_df,'2017')
delitos18_df=delitos_cleaner(raw_delitos18_df,'2018')
delitos19_df=delitos_cleaner(raw_delitos19_df,'2019')
delitos20_df=delitos_cleaner(raw_delitos20_df,'2020')

## Combinación de datasets

In [13]:
denue_df=pd.concat([denue15_df, denue16_df, denue17_df,
    denue18_df, denue19_df, denue20_df])
delitos_df=pd.concat([delitos15_df, delitos16_df, delitos17_df,
    delitos18_df, delitos19_df, delitos20_df])

## Exportación a parquet

In [25]:
shapes_gdf.to_parquet('tidy_data/son_shapes.parquet',index=False)
pobl_df.to_parquet('tidy_data/son_pobl.parquet',index=False)
denue_df.to_parquet('tidy_data/son_ue.parquet',index=False)
delitos_df.to_parquet('tidy_data/son_del.parquet',index=False)